# Dependencies

In [ ]:
!unzip /content/task2_images_resized.zip

In [ ]:
import numpy as np
from scipy.fftpack import dct, idct
from PIL import Image
import matplotlib.pyplot as plt
#from transformers import pipeline
import torch
import os
#from art.attacks.evasion import SimBA

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# pipe = pipeline("image-classification", model="dima806/ai_vs_real_image_detection", device=device)

# Functions

In [ ]:
def DCT_attack(x, epsilon=0.2, num_iters=1000):
    """
    Perform a DCT-based attack on an image in H, W, C format and return the attacked DCT coefficients and reconstructed image.

    Args:
        x (numpy.ndarray): Input image (shape: H, W, C), normalized.
        epsilon (float): Perturbation step size in the DCT domain.
        num_iters (int): Maximum number of perturbation iterations.

    Returns:
        attacked_dct (numpy.ndarray): Attacked DCT coefficients (flattened array).
        perturbed_image (numpy.ndarray): Reconstructed image from the attacked DCT (H, W, C).
    """
    # Validate input shape
    if len(x.shape) != 3:
        raise ValueError("Input image must have shape (H, W, C).")

    H, W, C = x.shape
    n_dims = H * W * C
    perm = np.random.permutation(n_dims)  # Random permutation of indices

    # Flatten the image for DCT manipulation
    x_flat = x.flatten()
    x_dct = dct(x_flat, norm='ortho')  # Convert to DCT domain

    # Attack in the DCT domain
    for i in range(num_iters):
        delta = np.zeros(n_dims)
        delta[perm[i]] = epsilon
        x_dct += delta

    # Convert back to the image domain
    perturbed_image = idct(x_dct, norm='ortho').reshape(H, W, C)
    #perturbed_image = (perturbed_image - np.min(perturbed_image)) / (np.max(perturbed_image) - np.min(perturbed_image))

    #return x_dct, perturbed_image
    return x_dct, perturbed_image

# Example Usage
# Assuming `input_image` is a numpy array of shape (H, W, C)

# input_image = ...  # Your numpy input (e.g., TinyImageNet image)
# attacked_dct, perturbed_image = DCT_attack(input_image, epsilon=0.2, num_iters=5000)
# print("Attacked DCT Shape:", attacked_dct.shape)  # Flattened shape: H*W*C
# print("Perturbed Image Shape:", perturbed_image.shape)  # Shape: (H, W, C)


def preprocess_DCT_attack(image, epsilon, num_iters):
  x = np.array(image)
  x_dct, perturbed_x = DCT_attack(x, epsilon, num_iters)
  perturbed_x = (perturbed_x - perturbed_x.min()) / (perturbed_x.max() - perturbed_x.min()) * 255
  perturbed_x = perturbed_x.astype(np.uint8)
  perturbed_image = Image.fromarray(perturbed_x)
  return perturbed_image

# TEST

In [ ]:
folder_path = "/content/task2_images_resized"
new_path = "/content/task2_images_resized_DCTattack"

count=0

In [ ]:
for filename in os.listdir(folder_path):
  count += 1
  if(count%100 == 0):
    print(f"On count: {count}")
  file_path = os.path.join(folder_path, filename)
  img = Image.open(file_path)
  if(count <= 1000):
    img = preprocess_DCT_attack(img, 4, 2000)
  elif(count <= 2000):
    img = preprocess_DCT_attack(img, 8, 2000)
  elif(count <= 3000):
    img = preprocess_DCT_attack(img, 12, 2000)
  elif(count <= 4000):
    img = preprocess_DCT_attack(img, 16, 2000)
  else:
    img = preprocess_DCT_attack(img, 20, 2000)
  target = os.path.join(new_path, filename)
  img.save(target)
  #ans = pipe(img)[0]['label']
  #if(ans == 'FAKE'):
  #  i += 1

#100*i/5000

In [ ]:
!zip -r /content/task2_images_resized_DCTattack.zip /content/task2_images_resized_DCTattack